# 计算图

计算图是用于神经网络BP算法的一种工具，其基本思想是复合函数的链式求导法则，可简化误差反向传播的计算。

## 局部计算节点

计算图将神经网络的推理与学习任务分散各个局部计算节点，通过局部节点的计算结果实现神经网络的推理与学习任务。

- **加法节点**

加法节点的作用是实现以下推理的计算片断，

$$
x + y = z
$$(node-add)

误差的反向传播则将$\frac{\partial L}{\partial z} $乘上以下局部计算结果后，

$$
\begin{split}
\frac{\partial z}{\partial x}&=1\\
\frac{\partial z}{\partial y}&=1\\
\end{split}
$$(node-add-local-comp)

反向传入相应分支，即，$x,y$的各分支分别反向传入$\frac{\partial L}{\partial z}\times 1$。式{eq}`node-add-local-comp`分别对应各个分支的局部梯度计算结果。

- **乘法节点**

与加法节点类似，实现以下局部计算，

$$
x*y=z
$$(node-mult)

误差反向传播则分别将以下结果反向传入对应分支，即$x$分支传入，

$$
\frac{\partial L}{\partial z}\frac{\partial z}{\partial x}=\frac{\partial L}{\partial z}\cdot y
$$(node-mult-back-x)

$y$分支传入，

$$
\frac{\partial L}{\partial z}\frac{\partial z}{\partial y}=\frac{\partial L}{\partial z}\cdot x
$$(node-mult-back-y)

- **分支节点**

分支节点是指相同的值复制后传入各个分支，也称为复制节点。反向传播则是上游传来的梯度之和。当分支扩展到$N$个节点，则可称为**重复节点**。重复节点的反向传播与分支节点类似，是上游所传的梯度之和。



In [12]:
import numpy as np
x = np.random.randn(1, 3)  # 假设的输入数据
y = np.repeat(x, 2, axis=0) # 正向传播

dy=np.random.randn(2, 3) #假设的梯度
dx=np.sum(dy, axis=0, keepdims=True) #梯度传播
print("Input x:\n", x)
print("Output y:\n", y) 
print("Gradient dy:\n", dy)
print("Gradient dx:\n", dx)

Input x:
 [[ 1.19485093  1.66341524 -1.59975205]]
Output y:
 [[ 1.19485093  1.66341524 -1.59975205]
 [ 1.19485093  1.66341524 -1.59975205]]
Gradient dy:
 [[-0.06655354  0.16201839 -0.75654729]
 [-1.0893431   0.20312229 -0.33846025]]
Gradient dx:
 [[-1.15589664  0.36514068 -1.09500754]]


- **sum节点**

sum节点与重复节点正好相反，推理时其输入为各个分支的和，反向传播时各分支传入的值是其上游值的复制。

In [13]:
x = np.random.randn(2, 3)  # 假设的输入数据
y = np.sum(x, axis=0, keepdims=True)  # 正向传播
dy = np.random.randn(1, 3)  # 假设的梯度
dx = np.repeat(dy, 2, axis=0)  # 梯度传播
print("Input x:\n", x)  
print("Output y:\n", y)
print("Gradient dy:\n", dy)
print("Gradient dx:\n", dx)

Input x:
 [[ 1.57649617 -0.46032091  1.06130582]
 [ 1.02455351 -0.34278245  0.93201846]]
Output y:
 [[ 2.60104969 -0.80310336  1.99332428]]
Gradient dy:
 [[-1.7463373  -0.01989783  0.34527752]]
Gradient dx:
 [[-1.7463373  -0.01989783  0.34527752]
 [-1.7463373  -0.01989783  0.34527752]]


- **MatMul节点**

矩阵乘积节点(假设向量为行向量)，即

$$
\pmb{y}_{1\times H}=\pmb{x}_{1\times D}\pmb{W}_{D\times H}
$$(node-matmul)

该计算结点的难点在于反向传播，即以下偏导数(**分子布局**)的计算，

$$
\begin{split}
\frac{\partial L}{\partial \pmb{x}}_{1\times D}&=\frac{\partial L}{\partial \pmb{y}}\frac{\partial \pmb{y}}{\partial \pmb{x}}   =\left[\frac{\partial L}{\pmb{y}}\right]_{1\times H}\left[\pmb{W}^\top\right]_{H\times D}\\
\left[\frac{\partial L}{\partial \pmb{W}}\right]_{D\times H}&=\frac{\partial L}{\partial \pmb{y}}\frac{\partial\pmb{y}}{\partial \pmb{W}}   =\left[\pmb{x}^\top\right]_{D\times 1} \left[\frac{\partial L}{\partial\pmb{y}}\right]_{1\times H}\\
\end{split}
$$(node-matmul-back)

式{eq}`node-matmul-back`的第1个等式容易实现，略过。第2个等式的推导如下：由矩阵乘法定义可知，

$$
\frac{\partial y_j}{\partial W_{ik}}=\left\{\begin{array}{ll}x_i,&j==k\\ 0,& j\neq k \end{array} \right.
$$

因此，可以得到以下等式，

$$
\frac{\partial L}{\partial W_{ij}}=\sum_k \frac{\partial L}{\partial y_k}\frac{\partial y_k}{\partial W_{ij}}=\frac{\partial L}{\partial y_j}\frac{\partial y_j}{\partial W_{ij}}=\frac{\partial L}{\partial y_j}\cdot x_i
$$

则有梯度如下，

$$
\frac{\partial L}{\partial \pmb{W}}=\left[ \frac{\partial L}{\partial y_j}x_i  \right]_{ij}=\left[\pmb{x}^\top\right]_{D\times 1} \left[\frac{\partial L}{\partial\pmb{y}}\right]_{1\times H}
$$

注意：当$\pmb{x}$是小批量样本时，{eq}`node-matmul-back`形式仍然保持不变。

In [14]:
class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]  #分子布局
        self.x = None
    
    def forward(self, x):
        W, = self.params
        out = np.dot(x, W)  # 矩阵乘法
        self.x = x  #保存输入，反向传播时使用
        return out
    
    def backward(self, dout):
        W, = self.params
        dx = np.dot(dout, W.T)  # 输入梯度
        dW = np.dot(self.x.T, dout)  # 权重梯度
        self.grads[0][...] = dW  # 更新梯度
        return dx
    
# 测试 MatMul 类
W = np.random.randn(3, 4)  # 假设的权重 
x = np.random.randn(2, 3)  # 假设的输入数据
matmul = MatMul(W)  
y = matmul.forward(x)  # 正向传播
dy = np.random.randn(2, 4)  # 假设的梯度    
dx = matmul.backward(dy)  # 反向传播
print("Input x:\n", x)  
print("Weights W:\n", W)
print("Output y:\n", y)     
print("Gradient dy:\n", dy)
print("Gradient dx:\n", dx)     
print("Gradient dW:\n", matmul.grads[0])  # 权重梯度
    
        

Input x:
 [[-0.38569833 -0.07324896 -0.58229009]
 [-0.96699073 -0.56693264  0.80395212]]
Weights W:
 [[ 0.13311926 -1.15183989  0.06628773 -0.69977611]
 [-0.45366951 -0.94971094  0.3307429  -0.43096151]
 [-1.43208512 -0.14871347  2.30186971  0.64263319]]
Output y:
 [[ 0.81577592  0.60042243 -1.39014957 -0.07272898]
 [-1.02285291  1.53268212  1.59898447  1.43764948]]
Gradient dy:
 [[ 2.26454874  0.3221979   0.40649511 -0.59622509]
 [ 0.05977523  1.20000979  2.68998884  0.31412483]]
Gradient dx:
 [[ 0.37450437 -0.94195614 -2.73839696]
 [-1.41576571 -0.41246162  6.12981002]]
Gradient dW:
 [[-0.93123475 -1.28466953 -2.75797876 -0.07379278]
 [-0.19976436 -0.70392538 -1.55481783 -0.13441475]
 [-1.27056787  0.77713777  1.92592416  0.59971728]]


## 局部计算的层

通过计算图的计算结点，可以实现一些神经网络的常用层。这些层一般都是结点的组合结果。

- **sigmoid层**

sigmoid层主要由sigmoid函数组成，即

$$
\begin{split}
y&=\frac{1}{\exp(-x)}\\
\frac{\partial y}{\partial x}&=y(1-y)
\end{split}
$$(sigmoid-layer)



In [15]:
class Sigmoid:
    def __init__(self):
        self.params = []    
        self.grads = []
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))  # Sigmoid 函数
        self.out = out  # 保存输出，反向传播时使用
        return out
    
    def backward(self, dout):
        dx = dout * self.out * (1-self.out)  # Sigmoid 的梯度
        return dx

# 测试 Sigmoid 类
x = np.random.randn(2, 3)  # 假设的输入数据
sigmoid = Sigmoid()
y = sigmoid.forward(x)  # 正向传播
dy = np.random.randn(2, 3)  # 假设的梯度
dx = sigmoid.backward(dy)  # 反向传播
print("Input x:\n", x)
print("Output y:\n", y)
print("Gradient dy:\n", dy)
print("Gradient dx:\n", dx)  # Sigmoid 的梯度


Input x:
 [[-0.02564088 -2.48760061 -1.30450939]
 [ 0.2162969  -0.28149647 -1.11479748]]
Output y:
 [[0.49359013 0.07673201 0.21340707]
 [0.55386439 0.43008693 0.24697757]]
Gradient dy:
 [[-1.6494359   0.5121364  -1.02999527]
 [ 0.67516035 -0.50797873 -0.81326976]]
Gradient dx:
 [[-0.41229121  0.0362819  -0.17289963]
 [ 0.1668312  -0.12451177 -0.15125163]]


- **仿射层(Affine)**